In [17]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import netCDF4 as nc

In [22]:
def Analyse(f1, f2, Latitude_range):
    fill_value = -32767.0
    
    nc_file_1 = nc.Dataset(f1, 'r')
    Raw_sst_1 = np.array(nc_file_1.variables['sst'][:])
    sst_1 = np.where(Raw_sst_1 == fill_value, np.nan, Raw_sst_1)
    latitudes = np.array(nc_file_1.variables['lat'][:])
    longitudes = np.array(nc_file_1.variables['lon'][:])
    nc_file_1.close()

    lat_indices = np.where((latitudes >= Latitude_range[0]) & (latitudes <= Latitude_range[1]))[0]
    lon_indices = np.where((longitudes >= 30) & (longitudes <= 120))[0]
    
    sst_1 = sst_1[lat_indices, :][:, lon_indices]

    nc_file_2 = nc.Dataset(f2, 'r')
    Raw_sst_2 = np.array(nc_file_2.variables['sst'][:])
    sst_2 = np.where(Raw_sst_2 == fill_value, np.nan, Raw_sst_2)
    nc_file_2.close()

    sst_2 = sst_2[lat_indices, :][:, lon_indices]

    mask = ~np.isnan(sst_1) & ~np.isnan(sst_2)
    sst_2 = sst_2[mask]
    sst_1 = sst_1[mask]

    mean_sst_1 = np.mean(sst_1)
    sst_1_Anomaly = sst_1 - mean_sst_1
    mean_sst_2 = np.mean(sst_2)
    sst_2_Anomaly = sst_2 - mean_sst_2

    data1 = sst_1_Anomaly
    data2 = sst_2_Anomaly
    return data1,data2

In [34]:
def linear_regress(d1, d2):
    d1 = np.array(d1)
    d2 = np.array(d2)
    
    w = 100
    b = 100
    m = len(d1)
    alpha = 0.1
    
    for i in range(3000):  
        y_pred = w * d1 + b
        
        error_fn = np.sum((y_pred - d2)**2) / (2 * m)
        
        print(f"Iteration {i+1}: w = {w}, b = {b}, Error = {error_fn}")
        
        w_gradient = np.sum((y_pred - d2) * d1) / m
        b_gradient = np.sum(y_pred - d2) / m
        
        w = w - alpha * w_gradient
        b = b - alpha * b_gradient
        
    return w, b

d1,d2 = Analyse('AQUA_MODIS.20040101_20041231.L3m.YR.SST.sst.4km.nc', 'AQUA_MODIS.20130101_20131231.L3m.YR.SST.sst.4km.nc',(-30,30))
w, b = linear_regress(d1, d2)
print(f"Final weights: w = {w}, b = {b}")

Iteration 1: w = 100, b = 100, Error = 33455.55054238282
Iteration 2: w = 42.51970244164232, b = 90.00000641370873, Error = 9056.418958545748
Iteration 3: w = 18.409828820000204, b = 81.00000811145516, Error = 4161.401599088499
Iteration 4: w = 8.297035451775036, b = 72.90000707394346, Error = 2812.2827653271133
Iteration 5: w = 4.0552646645205845, b = 65.61000236741307, Error = 2179.714457914281
Iteration 6: w = 2.2760724334082494, b = 59.04900394160129, Error = 1748.3046851199128
Iteration 7: w = 1.5297931861890164, b = 53.144104528361325, Error = 1413.1079078199143
Iteration 8: w = 1.216772980176919, b = 47.82969679191948, Error = 1144.1041058111032
Iteration 9: w = 1.085472713913687, b = 43.04672594316888, Error = 926.6521553362528
Iteration 10: w = 1.0304024826240317, b = 38.742054027950566, Error = 750.5935264928614
Iteration 11: w = 1.0073030272233587, b = 34.86784760650765, Error = 607.9998490892062
Iteration 12: w = 0.9976132278969868, b = 31.381062015847522, Error = 492.50135